In [1]:
import pandas as pd
import numpy as np

/Users/stoneman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/stoneman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
items = pd.read_csv('/Users/stoneman/Library/CloudStorage/OneDrive-Vanderbilt/Social_Network_Analysis/Amazon_Recommandation_Project/data/products_train_UK_representations.csv')

In [4]:
items.dtypes

index                 int64
id                   object
price               float64
representation       object
aggregated_price    float64
dtype: object

In [5]:
items.head()

,index,id,price,representation,aggregated_price
0,0,B07PNBPVN1,3.75,"[-0.21872417628765106, -0.2500726580619812, -0...",7.297000
1,1,B0B45XQ6Q3,8.59,"[-0.7130799889564514, -0.5445948839187622, -0....",11.618000
2,2,B01CR9IRWY,16.00,"[-0.8353576064109802, -0.6093693375587463, -0....",16.546154
3,3,B077RCBS9C,36.98,"[-0.8522624969482422, -0.5257986783981323, -0....",16.237500
4,4,B088BMDXPD,7.99,"[-0.8538389205932617, -0.47541171312332153, -0...",13.237500


In [6]:
representation_list = items['representation'].apply(eval).tolist()

# Convert the list of lists to an array where each row is the representation
representations_array = np.array(representation_list)

representations_array

array([[-0.21872418, -0.25007266, -0.8676973 , ..., -0.81096423,
        -0.48364848,  0.23691408],
       [-0.71307999, -0.54459488, -0.96643782, ..., -0.9660297 ,
        -0.63595694,  0.42050147],
       [-0.83535761, -0.60936934, -0.97644138, ..., -0.89264345,
        -0.69918853,  0.58237803],
       ...,
       [-0.85379964, -0.55762357, -0.90438592, ..., -0.87588161,
        -0.65999103,  0.67038369],
       [-0.65839285, -0.34608755, -0.97733754, ..., -0.97883922,
        -0.53299916,  0.49857637],
       [-0.88605624, -0.65183997, -0.99130929, ..., -0.98189676,
        -0.73334104,  0.73316491]])

In [10]:
representations_array.shape

(6690, 768)

In [6]:
# Map items to indices
item_to_index = {item_id: index for index, item_id in enumerate(items['id'])}


In [11]:
df_index = pd.read_csv('/Users/stoneman/Library/CloudStorage/OneDrive-Vanderbilt/Social_Network_Analysis/Amazon_Recommandation_Project/data/train_and_test_index.csv')

## Adjacency Matrix

In [12]:
# Initialize an adjacency matrix with zeros
adjacency_matrix = np.zeros((len(items), len(items)))

# Iterate over the training dataset to populate the adjacency matrix
for _, row in df_index.iterrows():

    adjacency_matrix[row['last_item_index'], row['next_item_index']] += 1  # Increment the weight for the link
    adjacency_matrix[row['last_second_index'], row['last_item_index']] += 1  # Increment the weight for the link

# transpose the matrix because I want to aggregate the features from the previous items
adjacency_matrix = adjacency_matrix.T

# Sum the elements of each row (axis=1 for rows)
row_sums = adjacency_matrix.sum(axis=1)

# Avoid division by zero by replacing zero row sums with 1
row_sums[row_sums == 0] = 1

# Divide each row by its sum to normalize it so that each row sums to 1
adjacency_matrix_normalized = adjacency_matrix / row_sums[:, np.newaxis]

### Aggregating the Features
Now we have the adjacency matrix, we can aggregate features. Since we're interested in the likelihood of purchasing the next_item based on prev_items, we want to collect features from source nodes (prev_items) to destination nodes (next_item). Thus, we should use A*X:

**Note: here we use the transpose of A, which is A.T, because we want to aggregate features from source nodes to destination nodes.**

In [16]:
# Now the row-normalized adjacency_matrix can be used to perform the aggregation
aggregated_representation = adjacency_matrix_normalized.dot(representations_array)

In [17]:
aggregated_representation.shape

(6690, 768)

In [23]:
combined_representation = np.hstack((representations_array, aggregated_representation))
combined_representation.shape

(6690, 1536)

In [25]:
combined_price = items[['price', 'aggregated_price']].values
combined_price.shape

(6690, 2)

In [1]:
items['price'].describe()

NameError: name 'items' is not defined

In [26]:
combined_features = np.hstack((combined_representation, combined_price))

In [27]:
combined_features.shape

(6690, 1538)

In [28]:
# Save the array to a file 
np.save('/Users/stoneman/Library/CloudStorage/OneDrive-Vanderbilt/Social_Network_Analysis/Amazon_Recommandation_Project/data/combined_features.npy', combined_features)

In [29]:
items

,index,id,price,representation,aggregated_price
0,0,B07PNBPVN1,3.75,"[-0.21872417628765106, -0.2500726580619812, -0...",7.297000
1,1,B0B45XQ6Q3,8.59,"[-0.7130799889564514, -0.5445948839187622, -0....",11.618000
2,2,B01CR9IRWY,16.00,"[-0.8353576064109802, -0.6093693375587463, -0....",16.546154
3,3,B077RCBS9C,36.98,"[-0.8522624969482422, -0.5257986783981323, -0....",16.237500
4,4,B088BMDXPD,7.99,"[-0.8538389205932617, -0.47541171312332153, -0...",13.237500
...,...,...,...,...,...
6685,6685,B00NBR70DO,13.22,"[-0.9247933626174927, -0.5451830625534058, -0....",47.385000
6686,6686,B08HYPQ4VR,12.99,"[-0.9244785308837891, -0.5655009150505066, -0....",9.966667
6687,6687,B09LR8NRL9,2.89,"[-0.8537996411323547, -0.557623565196991, -0.9...",4.481667
6688,6688,B0987DMWKW,15.99,"[-0.6583928465843201, -0.3460875451564789, -0....",14.990000


In [30]:
items_with_combined_features = items[['index', 'id']].copy()

In [31]:
items_with_combined_features['combined_features']=combined_features.tolist()

In [32]:
items_with_combined_features

,index,id,combined_features
0,0,B07PNBPVN1,"[-0.21872417628765106, -0.2500726580619812, -0..."
1,1,B0B45XQ6Q3,"[-0.7130799889564514, -0.5445948839187622, -0...."
2,2,B01CR9IRWY,"[-0.8353576064109802, -0.6093693375587463, -0...."
3,3,B077RCBS9C,"[-0.8522624969482422, -0.5257986783981323, -0...."
4,4,B088BMDXPD,"[-0.8538389205932617, -0.47541171312332153, -0..."
...,...,...,...
6685,6685,B00NBR70DO,"[-0.9247933626174927, -0.5451830625534058, -0...."
6686,6686,B08HYPQ4VR,"[-0.9244785308837891, -0.5655009150505066, -0...."
6687,6687,B09LR8NRL9,"[-0.8537996411323547, -0.557623565196991, -0.9..."
6688,6688,B0987DMWKW,"[-0.6583928465843201, -0.3460875451564789, -0...."


In [33]:
items_with_combined_features.to_csv('/Users/stoneman/Library/CloudStorage/OneDrive-Vanderbilt/Social_Network_Analysis/Amazon_Recommandation_Project/data/items_with_combined_features.csv', index=False)